In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from matplotlib import colors 
from matplotlib.ticker import PercentFormatter 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import os

print(os.listdir("../input"))

abnormal = pd.read_csv("../input/ptbdb_abnormal.csv", header = None) 
normal = pd.read_csv("../input/ptbdb_normal.csv", header = None)

abnormal = abnormal.drop([187], axis=1)
normal = normal.drop([187], axis=1)

abnormal.head

In [ ]:
flatten_ab_y = (abnormal.values)
flatten_ab_y  = flatten_ab_y[:,5:70].flatten()

print(flatten_ab_y.shape)

ab_x=np.arange(0,65)
ab_x = np.tile(ab_x, abnormal.shape[0])

plt.hist2d(ab_x, flatten_ab_y, bins = (65,100), cmap = plt.cm.jet) 

plt.show()

**From the above histogram color map for PTB data marked as abnormal, you can infer that most of the ECG features is widely distributed in the range of 0 - 0.4 and there is no fixed pattern to this data**

In [ ]:
plt.plot((abnormal.values)[0][5:70])
plt.show()

plt.plot((abnormal.values)[50][5:70])
plt.show()

plt.plot((abnormal.values)[117][5:70])
plt.show()

plt.plot((abnormal.values)[1111][5:70])
plt.show()

plt.plot((abnormal.values)[100][5:70])
plt.show()

**From the above, you can see that PTB data marked as abnormal donot have any fixed pattern in the data**

In [ ]:
flatten_norm_y = normal.values
flatten_norm_y  = flatten_norm_y[:,5:70].flatten()

norm_x=np.arange(0,65)
norm_x = np.tile(norm_x, normal.shape[0])

plt.hist2d(norm_x,flatten_norm_y, bins=(65,100), cmap=plt.cm.jet)
plt.show()

**From the above histogram color map for PTB data marked as normal, you can infer that the graph of ECG features follow a standard bell shape and they peak in between the features 20-30**

In [ ]:
plt.plot((normal.values)[0][5:70])
plt.show()

plt.plot((normal.values)[50][5:70])
plt.show()

plt.plot((normal.values)[117][5:70])
plt.show()

plt.plot((normal.values)[1111][5:70])
plt.show()

plt.plot((normal.values)[100][5:70])
plt.show()

**From the above, you can see that PTB data marked as normal has a fixed bell shape in the data and it peaks between the features 20-30 **

In [ ]:
y_abnormal = np.ones((abnormal.shape[0]))
y_abnormal = pd.DataFrame(y_abnormal)

y_normal = np.zeros((normal.shape[0]))
y_normal = pd.DataFrame(y_normal)

X = pd.concat([abnormal, normal], sort=True)
y = pd.concat([y_abnormal, y_normal] ,sort=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(abnormal.dtypes, normal.dtypes)

In [ ]:
abnormal.shape

In [ ]:
normal.shape

**Check any of the features have a null**

In [ ]:
np.any(X_train.isna().sum())

In [ ]:
np.any(X_test.isna().sum())

In [ ]:
# Create sequential model 
model = tf.keras.models.Sequential()

#Dense layer as first layer with 10 neurons, input share (187,) and and leaky Relu activation
model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape=(187,)))

#Dense layer as second layer with 10 neurons and leaky Relu activation
model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Dense layer as third layer with 10 neurons and leaky Relu activation
model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Dense layer as fourth layer with 10 neurons and leaky Relu activation
model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Softmax as last layer with two outputs
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
ann_model_history = model.fit(X_train, y_train, epochs=300, batch_size = 10, validation_data = (X_test, y_test))

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout


#Reshape train and test data to (n_samples, 187, 1), where each sample is of size (187, 1)
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)

# Create sequential model 
clf = tf.keras.models.Sequential()

#First CNN layer  with 32 filters, conv window 5, relu activation and same padding
clf.add(Conv1D(filters=32, kernel_size=(5,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train.shape[1],1)))

#Second CNN layer  with 64 filters, conv window 5, relu activation and same padding
clf.add(Conv1D(filters=64, kernel_size=(5,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Third CNN layer with 64 filters, conv window 5, relu activation and same padding
clf.add(Conv1D(filters=128, kernel_size=(5,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Fourth CNN layer with Max pooling
clf.add(MaxPool1D(pool_size=(5,), strides=2, padding='same'))
clf.add(Dropout(0.5))

#Flatten the output
clf.add(Flatten())

#Add a dense layer with 512 neurons
clf.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Add a dense layer with 1024 neurons
clf.add(Dense(units = 1024, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Softmax as last layer with two outputs
clf.add(Dense(units = 2, activation='softmax'))

In [ ]:
clf.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
clf.summary()

In [ ]:
history = clf.fit(X_train, y_train, epochs = 10)
y_pred = clf.predict(X_test)